In [1]:
from autocode import Optimization

optimization: Optimization = Optimization(
    num_cpus=12,
    server_host="0.0.0.0",
    server_port=10000,
    dashboard_port=10001,
)

2024-08-06 19:56:02,398	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at http://172.25.4.192:8265 


Available resources:  {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'node:172.25.4.192': 1.0, 'CPU': 12.0, 'object_store_memory': 4375123968.0, 'memory': 8750247936.0, 'GPU': 1.0}
Server is healthy.


In [2]:
import dotenv
from autocode import OptimizationUseCase, ApplicationSetting
from autocode.container import ApplicationContainer

dotenv.load_dotenv(dotenv.find_dotenv())
application_container: ApplicationContainer = ApplicationContainer()
application_setting: ApplicationSetting = application_container.settings.application()
application_setting.num_cpus = 12
application_setting.server_host = "0.0.0.0"
application_setting.server_port = 10000
application_setting.dashboard_port = 10001
optimization_use_case: OptimizationUseCase = application_container.use_cases.optimization()
# optimization_use_case.reset()

In [3]:
from python_on_whales import DockerClient
from typing import List

docker_clients: List[DockerClient] = optimization.deploy(
    compose_files=["./client-compose.yml"],
)

#0 building with "default" instance using docker driver

#1 [app-account internal] load build definition from .Dockerfile
#1 transferring dockerfile: 119B 0.0s done
#1 DONE 0.0s

#2 [app-gateway internal] load build definition from .Dockerfile
#2 transferring dockerfile: 119B 0.0s done
#2 DONE 0.0s

#3 [app-product internal] load build definition from .Dockerfile
#3 transferring dockerfile: 119B 0.0s done
#3 DONE 0.0s

#4 [app-product internal] load metadata for docker.io/library/golang:latest
#4 DONE 2.1s

#5 [app-gateway internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-product internal] load .dockerignore
#6 transferring context: 2B 0.0s done
#6 DONE 0.0s

#7 [app-account 1/5] FROM docker.io/library/golang:latest@sha256:86a3c48a61915a8c62c0e1d7594730399caa3feb73655dfe96c7bc17710e96cf
#7 DONE 0.0s

#8 [app-account internal] load .dockerignore
#8 transferring context: 2B 0.0s done
#8 DONE 0.0s

#9 [app-gateway internal] load build context
#9 tran

 Container autocode-worker-0-app-product-1  Created
 Container autocode-worker-0-app-gateway-1  Created
 Container autocode-worker-0-app-account-1  Created
 Container autocode-worker-0-app-product-1  Starting
 Container autocode-worker-0-app-gateway-1  Starting
 Container autocode-worker-0-app-account-1  Starting
 Container autocode-worker-0-app-account-1  Started
 Container autocode-worker-0-app-gateway-1  Started
 Container autocode-worker-0-app-product-1  Started


#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 119B 0.0s done
#1 DONE 0.0s

#2 [app-gateway internal] load build definition from .Dockerfile
#2 transferring dockerfile: 119B 0.0s done
#2 DONE 0.0s

#3 [app-account internal] load build definition from .Dockerfile
#3 transferring dockerfile: 119B 0.0s done
#3 DONE 0.0s

#4 [app-account internal] load metadata for docker.io/library/golang:latest
#4 DONE 0.3s

#5 [app-product internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-account 1/5] FROM docker.io/library/golang:latest@sha256:86a3c48a61915a8c62c0e1d7594730399caa3feb73655dfe96c7bc17710e96cf
#6 DONE 0.0s

#7 [app-gateway internal] load .dockerignore
#7 transferring context: 2B 0.0s done
#7 DONE 0.0s

#8 [app-account internal] load .dockerignore
#8 transferring context: 2B 0.0s done
#8 DONE 0.0s

#9 [app-gateway internal] load build context
#9 tran

 Container autocode-worker-1-app-account-1  Created
 Container autocode-worker-1-app-product-1  Created
 Container autocode-worker-1-app-gateway-1  Created
 Container autocode-worker-1-app-account-1  Starting
 Container autocode-worker-1-app-product-1  Starting
 Container autocode-worker-1-app-gateway-1  Starting
 Container autocode-worker-1-app-product-1  Started
 Container autocode-worker-1-app-account-1  Started
 Container autocode-worker-1-app-gateway-1  Started


#0 building with "default" instance using docker driver

#1 [app-gateway internal] load build definition from .Dockerfile
#1 transferring dockerfile: 119B 0.0s done
#1 DONE 0.0s

#2 [app-product internal] load build definition from .Dockerfile
#2 transferring dockerfile: 119B 0.0s done
#2 DONE 0.0s

#3 [app-account internal] load build definition from .Dockerfile
#3 transferring dockerfile: 119B 0.0s done
#3 DONE 0.0s

#4 [app-account internal] load metadata for docker.io/library/golang:latest
#4 DONE 0.4s

#5 [app-gateway internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-account internal] load .dockerignore
#6 transferring context: 2B 0.0s done
#6 DONE 0.0s

#7 [app-account 1/5] FROM docker.io/library/golang:latest@sha256:86a3c48a61915a8c62c0e1d7594730399caa3feb73655dfe96c7bc17710e96cf
#7 DONE 0.0s

#8 [app-product internal] load .dockerignore
#8 transferring context: 2B 0.0s done
#8 DONE 0.0s

#9 [app-gateway internal] load build context
#9 tran

...

 Network autocode-worker-9_default  Creating
 Network autocode-worker-9_default  Created
 Container autocode-worker-9-app-account-1  Creating
 Container autocode-worker-9-app-product-1  Creating
 Container autocode-worker-9-app-gateway-1  Creating
 Container autocode-worker-9-app-gateway-1  Created
 Container autocode-worker-9-app-account-1  Created
 Container autocode-worker-9-app-product-1  Created
 Container autocode-worker-9-app-gateway-1  Starting
 Container autocode-worker-9-app-product-1  Starting
 Container autocode-worker-9-app-account-1  Starting
 Container autocode-worker-9-app-product-1  Started
 Container autocode-worker-9-app-account-1  Started
 Container autocode-worker-9-app-gateway-1  Started


#0 building with "default" instance using docker driver

#1 [app-gateway internal] load build definition from .Dockerfile
#1 transferring dockerfile: 119B 0.0s done
#1 DONE 0.0s

#2 [app-product internal] load build definition from .Dockerfile
#2 transferring dockerfile: 119B 0.0s done
#2 DONE 0.0s

#3 [app-account internal] load build definition from .Dockerfile
#3 transferring dockerfile: 119B 0.0s done
#3 DONE 0.0s

#4 [app-account internal] load metadata for docker.io/library/golang:latest
#4 DONE 0.8s

#5 [app-gateway internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-account 1/5] FROM docker.io/library/golang:latest@sha256:86a3c48a61915a8c62c0e1d7594730399caa3feb73655dfe96c7bc17710e96cf
#6 DONE 0.0s

#7 [app-product internal] load .dockerignore
#7 transferring context: 2B 0.0s done
#7 DONE 0.0s

#8 [app-account internal] load .dockerignore
#8 transferring context: 2B done
#8 DONE 0.0s

#9 [app-gateway internal] load build context
#9 transferr

 Network autocode-worker-10_default  Creating
 Network autocode-worker-10_default  Created
 Container autocode-worker-10-app-product-1  Creating
 Container autocode-worker-10-app-account-1  Creating
 Container autocode-worker-10-app-gateway-1  Creating
 Container autocode-worker-10-app-gateway-1  Created
 Container autocode-worker-10-app-account-1  Created
 Container autocode-worker-10-app-product-1  Created
 Container autocode-worker-10-app-account-1  Starting
 Container autocode-worker-10-app-gateway-1  Starting
 Container autocode-worker-10-app-product-1  Starting
 Container autocode-worker-10-app-account-1  Started
 Container autocode-worker-10-app-product-1  Started
 Container autocode-worker-10-app-gateway-1  Started


#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 119B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 119B 0.0s done
#2 DONE 0.0s

#3 [app-gateway internal] load build definition from .Dockerfile
#3 transferring dockerfile: 119B 0.0s done
#3 DONE 0.0s

#4 [app-gateway internal] load metadata for docker.io/library/golang:latest
#4 DONE 0.7s

#5 [app-gateway internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-product 1/5] FROM docker.io/library/golang:latest@sha256:86a3c48a61915a8c62c0e1d7594730399caa3feb73655dfe96c7bc17710e96cf
#6 DONE 0.0s

#7 [app-account internal] load .dockerignore
#7 transferring context: 2B 0.0s done
#7 DONE 0.0s

#8 [app-product internal] load .dockerignore
#8 transferring context: 2B 0.0s done
#8 DONE 0.0s

#9 [app-gateway internal] load build context
#9 tran

 Network autocode-worker-11_default  Creating
 Network autocode-worker-11_default  Created
 Container autocode-worker-11-app-gateway-1  Creating
 Container autocode-worker-11-app-account-1  Creating
 Container autocode-worker-11-app-product-1  Creating
 Container autocode-worker-11-app-gateway-1  Created
 Container autocode-worker-11-app-product-1  Created
 Container autocode-worker-11-app-account-1  Created
 Container autocode-worker-11-app-account-1  Starting
 Container autocode-worker-11-app-product-1  Starting
 Container autocode-worker-11-app-gateway-1  Starting
 Container autocode-worker-11-app-gateway-1  Started
 Container autocode-worker-11-app-account-1  Started
 Container autocode-worker-11-app-product-1  Started


In [4]:
from autocode import OptimizationObjective
from typing import Dict, Any, List
from autocode import OptimizationEvaluateRunResponse

objectives: List[OptimizationObjective] = [
    OptimizationObjective(
        type="minimize",
    ),

    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),

    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
]


def evaluator(inputs: List[OptimizationEvaluateRunResponse]) -> Dict[str, Any]:
    f_gateway: List[float] = []
    f_account: List[float] = []
    f_product: List[float] = []
    for input in inputs:
        if "gateway" in input.get_client().name:
            f_gateway = input.objectives
        elif "account" in input.get_client().name:
            f_account = input.objectives
        elif "product" in input.get_client().name:
            f_product = input.objectives

    output: Dict[str, Any] = {
        "F": f_gateway + f_account + f_product
    }

    return output


In [5]:
result = optimization.run(
    objectives=objectives,
    num_inequality_constraints=0,
    num_equality_constraints=0,
    evaluator=evaluator
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |     14 |             - |             -
     2 |      200 |     14 |  1.174194E+01 |         nadir
     3 |      300 |     14 |  0.0023041475 |             f
     4 |      400 |     14 |  0.0230414747 |             f
     5 |      500 |     14 |  0.0115207373 |             f
     6 |      600 |     14 |  4.1666666667 |         nadir
     7 |      700 |     14 |  0.2500000000 |         ideal
     8 |      800 |     13 |  0.3333333333 |         nadir
     9 |      900 |     14 |  0.5714285714 |         ideal
    10 |     1000 |     16 |  0.000000E+00 |             f
    11 |     1100 |     17 |  0.0252100840 |             f
    12 |     1200 |     18 |  0.000000E+00 |             f
    13 |     1300 |     18 |  0.0158730159 |             f
    14 |     1400 |     18 |  0.000000E+00 |             f
    15 |     1500 |     18 |  0.0079365079 |             f
    16 |     1600 |     18 |  0.000000E+00 |            